In [1]:
# import the libraries required

#numpy - lists, arrays, dictionaries, string/number functions
import numpy as np

#pandas - read/write csv file
import pandas as pd

#nsetools - works as a API to get stock details from NSE India website
import nsetools
from nsetools import Nse

nse = Nse()

#json - converts a string to a json/dictionary format
import json

# import the urllib library to access the website
#import urllib
#from urllib import request
import urllib.request

#regex - to perform findall function
import re

import requests

In [2]:
# read the input dataset using pandas function
input_file = pd.read_csv('NSE_Data.csv')

In [3]:
# get all the stock names from the input into an internal array
stock_list = input_file['Symbol']
print(stock_list)

0             NIFTY 50
1        NIFTY NEXT 50
2      NIFTY MIDCAP 50
3           ADANIPORTS
4           ASIANPAINT
            ...       
148         TORNTPHARM
149         TORNTPOWER
150           TVSMOTOR
151          UNIONBANK
152             VOLTAS
Name: Symbol, Length: 153, dtype: object


In [4]:
# intialize and create empty lists of all output values

temp_input_file = input_file.copy()

Company_Name = []
Last_Traded_Values = temp_input_file['Last Traded Price']
Change = temp_input_file['Change']
Change_Percent = temp_input_file['%Change']
Prev_Day_Close = []
Open_Price = temp_input_file['Open']
Day_Low = temp_input_file['Low']
Day_High = temp_input_file['High']
Low_52wk = temp_input_file['52 Week Low']
High_52wk = temp_input_file['52 Week High']
Date_52wk_Low = ['NA','NA','NA']
Date_52wk_High = ['NA','NA','NA']
Face_Value = ['NA','NA','NA']

Sectors = ['NA','NA','NA']
Industrys = ['NA','NA','NA']
RSIs = [0,0,0]
PEs = [0,0,0]
PBs = [0,0,0]
Div_Yields = [0,0,0]
Promoter_Stake = ['NA','NA','NA']
Promoter_Stake_Change = ['NA','NA','NA']

In [5]:
# This avoids the website to block the python program to get in and pretends to be a real user accessing it
Headers = {
           'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
           'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
           'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
           'Accept-Encoding': 'none',
           'Accept-Language': 'en-US,en;q=0.8',
           'Connection': 'keep-alive'
          }

In [6]:
def get_index_html_data():
    # connect the NSE India website and read the data on html page
    in_url = 'https://www.nseindia.com/live_market/dynaContent/live_watch/stock_watch/liveIndexWatchData.json'
    in_ref = 'https://www.nseindia.com/live_market/dynaContent/live_watch/live_index_watch.htm'
    NSE_Headers = {
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36 Edge/18.17763',
           'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
           'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
           'Accept-Encoding': 'gzip, deflate, br',
           'Accept-Language': 'en-US,en;q=0.8',
           'Connection': 'keep-alive',
           'referer' : in_ref
          }
    
    in_index_data_str = requests.get(in_url, headers = NSE_Headers).json()['data'][0:9]
    #print(in_index_data_str)
    return in_index_data_str

In [7]:
def get_index_details(in_index_data,in_index_name,index_pos):
# get the index details using NSE API and provide link to the NSE website page

    #print(in_index_name)
    #print(index_pos)
    
# move the values into the respective lists/columns
    in_company_name = in_index_data['indexName']
    #print("Company Name " + str(in_company_name))
    Company_Name.append(str(in_company_name))

    in_trade_price = in_index_data['last']
    #print("Last Traded Price " + str(in_trade_price))
    Last_Traded_Values[index_pos] = str(in_trade_price.replace(',',''))

    in_change_percent = in_index_data['percChange']
    #print("Today's Change % " + str(in_change_percent))
    Change_Percent[index_pos] = str(in_change_percent)

    in_prev_close = in_index_data['previousClose']
    #print("Previous Day's Close " + str(in_prev_close))
    Prev_Day_Close.append(str(in_prev_close.replace(',','')))

    in_open_price = in_index_data['open']
    #print("Open Price " + str(in_open_price))
    Open_Price[index_pos] = str(in_open_price.replace(',',''))

    in_day_low = in_index_data['low']
    #print("Day's Low " + str(in_day_low))
    Day_Low[index_pos] = str(in_day_low.replace(',',''))

    in_day_high = in_index_data['high']
    #print("Day's High " + str(in_day_high))
    Day_High[index_pos] = str(in_day_high.replace(',',''))

    in_52wk_low = in_index_data['yearLow']
    #print("52 Week Low " + str(in_52wk_low))
    Low_52wk[index_pos] = str(in_52wk_low.replace(',',''))

    in_52wk_high = in_index_data['yearHigh']
    #print("52 Week High " + str(in_52wk_high))
    High_52wk[index_pos] = str(in_52wk_high.replace(',',''))


    in_api_data = nse.get_index_quote(in_index_name)
    #print(in_api_data)

    in_day_change = in_api_data['change']
    #print("Today's Change " + str(in_day_change))
    Change[index_pos] = str(in_day_change.replace(',',''))
    
    

In [8]:
def get_html_data(url):
#   connect's the url passed to this function, read the html page and return the data on the page.

    url_request = urllib.request.Request(url, headers = Headers)
    html_page = urllib.request.urlopen(url_request)
    html_data = html_page.read()
    return html_data

In [9]:
def get_stock_details(in_stock_name,stock_pos):
# get the stock details from NSE India website and provide link to the page

# replace the '&' with '%26'. But not required for the HTML page view source search
    in_stock_name_modify = in_stock_name.replace("&","%26")
    
# connect the NSE India website and read the data on html page
    in_url = 'https://www.nseindia.com/live_market/dynaContent/live_watch/get_quote/GetQuote.jsp?symbol=' + in_stock_name_modify
    in_html_data = get_html_data(in_url)
    in_html_data_str = str(in_html_data)
    #print(in_html_data_str)
    
    if (len(in_html_data_str) > 0):
# get the string with required data from html data read
        in_search_for = '{"tradedDate"'
        in_search_for_end = ',"optLink":'
        in_search_string = in_html_data_str[in_html_data_str.find(in_search_for):in_html_data_str.find(in_search_for_end)] + str('}')
        in_search_string = in_search_string.replace(r"\'s","'s")
        #print(in_search_string)
        in_search_string_dict = json.loads(in_search_string)
        #print(in_search_string_dict)
        
        in_trade_date = in_search_string_dict['tradedDate']
        #print("Trade Date " + str(in_trade_date))
        
        in_search_string_data = in_search_string_dict['data'][0]
        #print(in_search_string_data)
    else:
        in_search_string_data = nse.get_quote(in_stock_name)
        
# move the values into the respective lists/columns
    in_company_name = in_search_string_data['companyName']
    #print("Company Name " + str(in_company_name))
    Company_Name.append(str(in_company_name))

    in_trade_price = in_search_string_data['lastPrice']
    #print("Last Traded Price " + str(in_trade_price))
    Last_Traded_Values[stock_pos] = str(in_trade_price.replace(',',''))
    
    in_day_change = in_search_string_data['change']
    #print("Today's Change " + str(in_day_change))
    Change[stock_pos] = str(in_day_change.replace(',',''))
    
    in_change_percent = in_search_string_data['pChange']
    #print("Today's Change % " + str(in_change_percent))
    Change_Percent[stock_pos] = str(in_change_percent)
    
    in_prev_close = in_search_string_data['previousClose']
    #print("Previous Day's Close " + str(in_prev_close))
    Prev_Day_Close.append(str(in_prev_close.replace(',','')))
    
    in_open_price = in_search_string_data['open']
    #print("Open Price " + str(in_open_price))
    Open_Price[stock_pos] = str(in_open_price.replace(',',''))
    
    in_day_low = in_search_string_data['dayLow']
    #print("Day's Low " + str(in_day_low))
    Day_Low[stock_pos] = str(in_day_low.replace(',',''))
    
    in_day_high = in_search_string_data['dayHigh']
    #print("Day's High " + str(in_day_high))
    Day_High[stock_pos] = str(in_day_high.replace(',',''))
        
    in_52wk_low = in_search_string_data['low52']
    #print("52 Week Low " + str(in_52wk_low))
    Low_52wk[stock_pos] = str(in_52wk_low.replace(',',''))
    
    in_52wk_high = in_search_string_data['high52']
    #print("52 Week High " + str(in_52wk_high))
    High_52wk[stock_pos] = str(in_52wk_high.replace(',',''))
    
    in_date_52wk_low_rchd = in_search_string_data['cm_adj_low_dt']
    #print("Date 52 Week Low Reached " + str(in_date_52wk_low_rchd))
    Date_52wk_Low.append(str(in_date_52wk_low_rchd))
    
    in_date_52wk_high_rchd = in_search_string_data['cm_adj_high_dt']
    #print("Date 52 Week High Reached " + str(in_date_52wk_high_rchd))
    Date_52wk_High.append(str(in_date_52wk_high_rchd))
    
    in_face_value = in_search_string_data['faceValue']
    #print("Face Value " + str(in_face_value))
    Face_Value.append(str(in_face_value))

In [10]:
def get_RSI(RSI_stock_name):
# get the current RSI value from traderscockpit and provide link to the page

# replace the '&' with '%26'. But not required for the HTML page view source search.
    RSI_stock_name_modify = RSI_stock_name.replace("&","%26")
 
    RSI_url = 'https://www.traderscockpit.com/?pageView=rsi-indicator-rsi-chart&type=rsi&symbol=' + RSI_stock_name_modify
    RSI_html_data = get_html_data(RSI_url)

    RSI_search_for = "is (.+?). <b>" + RSI_stock_name + " RSI analysis"
    RSI_com = re.compile(RSI_search_for)
    RSI_search_result = re.findall(RSI_com,str(RSI_html_data))

    if(RSI_search_result):
        RSI_value = RSI_search_result[0][-5:]
        RSIs.append(float(RSI_value))
        print(RSI_stock_name + ' found ' + RSI_value)
    else:
        RSIs.append(0)
#        print(RSI_stock_name + 'notfound')

In [11]:
def get_value(html_data_str,search_for,search_for_end):
#   search for the P/E and Div Yield from the input HTML data string and extract the sub-string required to find the values.

    search_result = html_data_str[html_data_str.find(search_for):html_data_str.find(search_for_end)]
    #print(search_result)
    search_result = search_result.replace(',','').replace(r'&amp;','&').replace(r'\n',' ').replace(r'</a>','<b>').replace(r'</b>','<b>').replace(r'<i class="icon-rupee"></i>',' ').replace(r'/">','<b>')
    search_result_edit = search_result.split('<b>')
    #print(search_result_edit)
    
    search_value = search_result_edit[1]
    final_value = search_value.strip()
    #print(final_value)
    return final_value

In [16]:
def get_Sect_PE_PB_Div_Yield(Sect_PE_PB_Div_stock_name,Sect_PE_PB_Div_curr_price):
# get the current Sector, P/E, P/B and Dividend Yield values from screener and provide link to the page

# connect to screener.in website with the stock name and read the webpage displayed through view source
    Sect_PE_PB_Div_url = 'https://www.screener.in/company/' + Sect_PE_PB_Div_stock_name + '/consolidated/'
    Sect_PE_PB_Div_html_data = get_html_data(Sect_PE_PB_Div_url)
    Sect_PE_PB_Div_html_data_str = str(Sect_PE_PB_Div_html_data)

# search for the parameter Sector. Gets the string between the strings provided.
    #print(Sect_PE_PB_Div_html_data_str)
    Sect_search_for = 'Sector:'
    Sect_search_for_end = 'Industry:'
    Sect_value = get_value(Sect_PE_PB_Div_html_data_str,Sect_search_for,Sect_search_for_end)
    #print(Sect_value)
    Sectors.append(Sect_value)

# search for the parameter Industry. Gets the string between the strings provided.
    #print(Sect_PE_PB_Div_html_data_str)
    Ind_search_for = 'Industry:'
    Ind_search_for_end = 'Loading peers'
    Ind_value = get_value(Sect_PE_PB_Div_html_data_str,Ind_search_for,Ind_search_for_end)
    #print(Ind_value)
    Industrys.append(Ind_value)

# search for the parameters Stock P/E. Gets the string between the strings provided.
    #print(Sect_PE_PB_Div_html_data_str)
    PE_search_for = 'Stock P/E:'
    PE_search_for_end = 'Dividend Yield:'
    PE_value = get_value(Sect_PE_PB_Div_html_data_str,PE_search_for,PE_search_for_end)
    #print(PE_value)
    PEs.append(PE_value)

# search for the parameter Dividend Yield %. Gets the string between the strings provided.
    #print(Sect_PE_PB_Div_html_data_str)
    Div_search_for = 'Dividend Yield:'
    Div_search_for_end = 'ROCE:'
    Div_Yield_value = get_value(Sect_PE_PB_Div_html_data_str,Div_search_for,Div_search_for_end)
    #print(Div_Yield_value)
    Div_Yields.append(Div_Yield_value)
    
# search for the parameter Book Value. Gets the string between the strings provided.
    #print(Sect_PE_PB_Div_html_data_str)
    PB_search_for = 'Book Value:'
    PB_search_for_end = 'Stock P/E:'
    PB_Book_value = get_value(Sect_PE_PB_Div_html_data_str,PB_search_for,PB_search_for_end)
    #print(PB_Book_value)
    if PB_Book_value.replace('.','').isnumeric():
        PB_value = round((float(Sect_PE_PB_Div_curr_price) / float(PB_Book_value)),2)
    else:
        PB_value = 0.00
    #print(PB_value)
    PBs.append(PB_value)

In [13]:
def get_promoter_stake(in_stock_name):
# get the promoter stake from NSE India website and provide link to the page

# replace the '&' with '%26'. But not required for the HTML page view source search
    in_stock_name_modify = in_stock_name.replace("&","%26")
    #print(in_stock_name_modify)
    
# connect the NSE India website and read the data on html page
    in_url = 'https://www.nseindia.com/live_market/dynaContent/live_watch/get_quote/companySnapshot/getShareholdingPattern' + in_stock_name_modify + '.json'
    in_ref = 'https://www.nseindia.com/live_market/dynaContent/live_watch/get_quote/GetQuote.jsp?symbol=' + in_stock_name_modify
    NSE_Headers = {
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36 Edge/18.17763',
           'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
           'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
           'Accept-Encoding': 'gzip, deflate, br',
           'Accept-Language': 'en-US,en;q=0.8',
           'Connection': 'keep-alive',
           'referer' : in_ref
          }
    in_promoter_stake = requests.get(in_url, headers = NSE_Headers).json()['holdingPercent'][0]
    #print("Promoter Stake " + str(in_promoter_stake))
    Promoter_Stake.append(str(in_promoter_stake))
    
    in_promoter_stake_prev = requests.get(in_url, headers = NSE_Headers).json()['holdingPercent'][4]
    if in_promoter_stake > in_promoter_stake_prev:
        in_promoter_stake_change = 'Increased'
    if in_promoter_stake < in_promoter_stake_prev:
        in_promoter_stake_change = 'Decreased'
    if in_promoter_stake == in_promoter_stake_prev:
        in_promoter_stake_change = 'No Change'
    
    #print("Promoter Stake Change " + str(in_promoter_stake_change))
    Promoter_Stake_Change.append(str(in_promoter_stake_change))


In [17]:
# for each stock name access websites to get the RSI (14), P/E, Debt-to-Equity Ratio, Book Value/Share, Price-to-Book value, Dividend-Yield %

index_html_data = get_index_html_data()
i = 0
for d in range(0,9):
    index_data = index_html_data[d]
    if stock_list[i] == index_data['indexName']:
        get_index_details(index_data,stock_list[i],i)
        i=i+1

for i in range(3,len(stock_list)):
#for i in range(58,62):
    get_stock_details(stock_list[i],i)
    get_RSI(stock_list[i])
    curr_price = Last_Traded_Values[i]
    print(curr_price)
    get_Sect_PE_PB_Div_Yield(stock_list[i],curr_price)
    #get_Sect_PE_PB_Div_Yield('CASTROLIND',1)
    get_promoter_stake(stock_list[i])

C:\Users\sureg\Miniconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\sureg\Miniconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\sureg\Miniconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\sureg\Miniconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A 

b'\n\n\n<!DOCTYPE html>\n<html lang="en">\n\n<head>\n  <!-- Basic Page Needs -->\n  <meta charset="utf-8">\n  <title>Castrol India Ltd - Screener</title>\n  <meta name="description" content="Screener is a stock screener and research tool for Indian stocks.">\n  <meta name="author" content="Mittal Analytics Private Limited">\n  \n  <!-- Mobile Specific Meta -->\n  <meta name="viewport" content="width=device-width, initial-scale=1">\n\n  <!-- Google Tag Manager -->\n  <script>(function(w,d,s,l,i){w[l]=w[l]||[];w[l].push({\'gtm.start\':\n  new Date().getTime(),event:\'gtm.js\'});var f=d.getElementsByTagName(s)[0],\n  j=d.createElement(s),dl=l!=\'dataLayer\'?\'&l=\'+l:\'\';j.async=true;j.src=\n  \'https://www.googletagmanager.com/gtm.js?id=\'+i+dl;f.parentNode.insertBefore(j,f);\n  })(window,document,\'script\',\'dataLayer\',\'GTM-KG8ZGC4\');</script>\n  <!-- End Google Tag Manager -->\n\n  <!-- FONT -->\n  <link rel="stylesheet" href="https://fonts.googleapis.com/css?family=Open+Sans:300,

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [72]:
out_file = {'Symbol' : stock_list,
            'Company Name' : Company_Name,
            'Sector' : Sectors,
            'Industry' : Industrys,
            'Current Price' : Last_Traded_Values,
            'Previous Day Close' : Prev_Day_Close,
            'Open Price' : Open_Price,
            'Day Low' : Day_Low,
            'Day High' : Day_High,
            'Day Change' : Change,
            'Day Change %' : Change_Percent,
            '52 Wk Low' : Low_52wk,
            '52 Wk High' : High_52wk,
            'Date 52Wk Low Reached' : Date_52wk_Low,
            'Date 52Wk High Reached' : Date_52wk_High,
            'Face Value' : Face_Value,
            'Div_Yield' : Div_Yields,
            'RSI' : RSIs,
            'PB' : PBs,
            'PE' : PEs,
            'Promoter Stake' : Promoter_Stake,
            'Promoter Stake Change' : Promoter_Stake_Change
           }

In [73]:
out_file_csv = pd.DataFrame(out_file)
out_file_csv.head(10)

,Symbol,Company Name,Sector,Industry,Current Price,Previous Day Close,Open Price,Day Low,Day High,Day Change,...,52 Wk High,Date 52Wk Low Reached,Date 52Wk High Reached,Face Value,Div_Yield,RSI,PB,PE,Promoter Stake,Promoter Stake Change
0,NIFTY 50,NIFTY 50,NA,NA,10704.80,10840.65,10845.20,10670.25,10845.20,-135.85,...,12103.05,NA,NA,NA,0,0.00,0.00,0,NA,NA
1,NIFTY NEXT 50,NIFTY NEXT 50,NA,NA,25922.10,26149.90,26178.70,25814.75,26189.00,-227.80,...,29496.50,NA,NA,NA,0,0.00,0.00,0,NA,NA
2,NIFTY MIDCAP 50,NIFTY MIDCAP 50,NA,NA,4183.50,4255.75,4259.70,4169.55,4262.85,-72.25,...,5137.45,NA,NA,NA,0,0.00,0.00,0,NA,NA
3,ADANIPORTS,Adani Ports and Special Economic Zone Limited,Infrastructure Developers & Operators,Miscellaneous,362.00,366.35,366.30,357.85,366.50,-4.35,...,430.60,28-JAN-19,23-MAY-19,2.00,0.06,42.65,3.06,16.90,62.30,No Change
4,ASIANPAINT,Asian Paints Limited,Paints/Varnish,Paints / Varnishes,1551.20,1547.75,1553.90,1545.00,1563.50,3.45,...,1622.50,23-OCT-18,30-AUG-19,1.00,0.68,52.22,15.63,66.08,52.79,No Change
5,AXISBANK,Axis Bank Limited,Banks,Banks - Private Sector,637.35,648.40,648.20,632.60,650.00,-11.05,...,827.75,26-OCT-18,04-JUN-19,2.00,0.15,38.57,2.46,33.10,18.70,Increased
6,BAJAJ-AUTO,Bajaj Auto Limited,Automobile,Automobiles - Scooters And 3 - Wheelers,2736.30,2777.05,2773.00,2721.45,2776.00,-40.75,...,3149.95,25-OCT-18,17-MAY-19,10.00,2.19,46.51,3.41,17.03,51.18,Decreased
7,BAJAJFINSV,Bajaj Finserv Limited,Finance,Finance & Investments,7195.00,7255.05,7246.00,7135.55,7249.00,-60.05,...,8580.00,24-OCT-18,05-JUL-19,5.00,0.03,48.73,4.82,35.32,58.37,Decreased
8,BAJFINANCE,Bajaj Finance Limited,Finance,Finance & Investments,3349.00,3413.85,3426.00,3340.30,3426.40,-64.85,...,3762.00,08-OCT-18,05-JUL-19,2.00,0.18,51.16,9.86,44.77,55.17,Decreased
9,BHARTIARTL,Bharti Airtel Limited,Telecomm-Service,Telecommunications - Service Provider,337.00,335.60,338.40,334.70,342.55,1.40,...,378.75,22-OCT-18,08-AUG-19,5.00,0.58,41.80,2.42,,62.71,No Change


In [74]:
out_file_csv.to_csv('Output.csv')

# These are backup functions

In [244]:
def get_PE(rsi_stock_name):
# get the current RSI value from screener and provide link to the page

    url = 'https://www.screener.in/company/' + rsi_stock_name + '/consolidated/'
    url_request = urllib.request.Request(url, headers = Headers)
    html_page = urllib.request.urlopen(url_request)
    html_data = html_page.read()
        
#    search_for = "Stock P/E:\n      \n      <b>"
#    com = re.compile(search_for)
#    print(com)
#    search_result = re.search(r'Stock P/E:*</p>', str(html_data), re.DOTALL)
#    search_result = re.findall('Stock P/E:\n      \n      <b>.*</b>\n      \n    \n  </li>\n  \n  <li class="four columns"',str(html_data),re.MULTILINE)
#    print(search_result)

    html_data_str = str(html_data)
    search_for = 'Stock P/E:'
    search_for_end = 'ROCE'
    search_result = html_data_str[html_data_str.find(search_for):html_data_str.find(search_for_end)]
#    my_list = [i for i in search_result.split("\n")]
#    print(my_list)
    search_result_edit = " ".join(search_result.split())
#    search_result_edit = search_result_edit.rstrip("n")

#    print(html_data[].split(search_for))

#        print('Stock Name: ' + rsi_stock_name + ' PE Value: ' + str(PE_value))
    
    print(search_result_edit)
#    print(type(search_result_edit[19:24]))
    if(search_result_edit[19:24].find(".") == 2 or search_result_edit[19:24].find(".") == 1):
        PE_value = search_result_edit[19:24]
        print(rsi_stock_name + 'found')
        if (search_result_edit[19:24].find(".") == 1):
            PE_value = search_result_edit[19:23]
        PEs.append(float(PE_value))
    else:
        PEs.append(0)
#        print(rsi_stock_name + 'notfound')

    if(search_result_edit[-65:-61].find(".") == 1 or search_result_edit[-65:-60].find(".") == 2):
        Div_Yield_value = search_result_edit[-65:-61]
        if (search_result_edit[-65:-61].find(".") == 2):
            Div_Yield_value = search_result_edit[-65:-60]
        Div_Yields.append(float(Div_Yield_value))
        print(rsi_stock_name + ' found ' + float(Div_Yield_value))
    else:
        Div_Yields.append(0)
#        print(rsi_stock_name + 'notfound')

In [ ]:
#   P/E and Div Yield can be in the format of XX.XX or X.XX or No value. Handled all scenarios.
#    if(search_result_edit[start_pos:end_pos].find(".") == 2 or search_result_edit[start_pos:end_pos].find(".") == 1):
#        search_value = search_result_edit[start_pos:end_pos]
#        if (search_result_edit[start_pos:end_pos].find(".") == 1):
#            search_value = search_result_edit[start_pos:end_pos-1]
#        #print('found')
#        return float(search_value)
#    else:
#        #print('not found')
#        return float(0)


dot_pos = search_result_edit[start_pos:end_pos].find(".")

if dot_pos == 0:
    0
else:
    end_pos - (6-dot_pos)

if = 6:
    then as is
if = 5:
    then end_pos-1
if = 4:
    then end_pos-2
if = 3:
    then end_pos-3
if = 2:
    then end_pos-4
if = 1:
    then end_pos-5

xxxxxx.xx
012345678
        search_value = search_result_edit[start_pos:end_pos]
        if (search_result_edit[start_pos:end_pos].find(".") == 1):
            search_value = search_result_edit[start_pos:end_pos-1]
        #print('found')
        return float(search_value)
    else:
        #print('not found')
        return float(0)

In [ ]:
def get_index_details(in_index_name,index_pos):
# get the index details using NSE API and provide link to the NSE website page

    print(in_index_name)
    
    in_api_data = nse.get_index_quote(in_index_name)
    print(in_api_data)
    
# move the values into the respective lists/columns
    in_company_name = in_api_data['name']
    #print("Company Name " + str(in_company_name))
    Company_Name[index_pos] = str(in_company_name)

    in_trade_price = in_api_data['lastPrice']
    #print("Last Traded Price " + str(in_trade_price))
    Last_Traded_Values[index_pos] = str(in_trade_price)
    
    in_day_change = in_api_data['change']
    #print("Today's Change " + str(in_day_change))
    Change[index_pos] = str(in_day_change)
    
    in_change_percent = in_api_data['pChange']
    #print("Today's Change % " + str(in_change_percent))
    Change_Percent[index_pos] = str(in_change_percent)

In [ ]:
def search_stock_details(search_for,search_string):
# get the "search_for" value from "search_string" passed

    com = re.compile(search_for)
    search_result = re.findall(com,search_string)
    return search_result

In [ ]:
def get_stock_details1(in_stock_name,stock_pos):
# get the stock details from NSE India website and provide link to the page

# replace the '&' with '%26'. But not required for the HTML page view source search
    in_stock_name_modify = in_stock_name.replace("&","%26")
    
# connect the NSE India website and read the data on html page
    in_url = 'https://www.nseindia.com/live_market/dynaContent/live_watch/get_quote/GetQuote.jsp?symbol=' + in_stock_name_modify
    in_html_data = get_html_data(in_url)
    in_html_data_str = str(in_html_data)
    #print(in_html_data_str)
    
# get the string with required data from html data read
    in_search_for = '{"tradedDate"'
    in_search_for_end = '"optLink":'
    in_search_string = in_html_data_str[in_html_data_str.find(in_search_for):in_html_data_str.find(in_search_for_end)]
    #print(in_search_string)
    
# move the values into the respective lists/columns
    in_search_for = '"tradedDate":"(.+?)"'
    in_trade_date = search_stock_details(in_search_for,in_search_string)
    #print("Trade Date " + str(in_trade_date))

    in_search_for = '"companyName":"(.+?)"'
    in_company_name = search_stock_details(in_search_for,in_search_string)
    print("Company Name " + str(in_company_name))
    Company_Name.append(str(in_company_name))
    
    in_search_for = '"lastPrice":"(.+?)"'
    in_trade_price = search_stock_details(in_search_for,in_search_string)
    #print("Last Traded Price " + str(in_trade_price))
    Last_Traded_Values[stock_pos] = str(in_trade_price)
    
    in_search_for = '"change":"(.+?)"'
    in_day_change = search_stock_details(in_search_for,in_search_string)
    #print("Today's Change " + str(in_day_change))
    Change[stock_pos] = str(in_day_change)
    
    in_search_for = '"pChange":"(.+?)"'
    in_change_percent = search_stock_details(in_search_for,in_search_string)
    #print("Today's Change % " + str(in_change_percent))
    Change_Percent[stock_pos] = str(in_change_percent)
    
    in_search_for = '"previousClose":"(.+?)"'
    in_prev_close = search_stock_details(in_search_for,in_search_string)
    #print("Previous Day's Close " + str(in_prev_close))
    Prev_Day_Close.append(str(in_prev_close))
    
    in_search_for = '"open":"(.+?)"'
    in_open_price = search_stock_details(in_search_for,in_search_string)
    #print("Open Price " + str(in_open_price))
    Open_Price[stock_pos] = str(in_open_price)
    
    in_search_for = '"dayLow":"(.+?)"'
    in_day_low = search_stock_details(in_search_for,in_search_string)
    #print("Day's Low " + str(in_day_low))
    Day_Low[stock_pos] = str(in_day_low)
    
    in_search_for = '"dayHigh":"(.+?)"'
    in_day_high = search_stock_details(in_search_for,in_search_string)
    #print("Day's High " + str(in_day_high))
    Day_High[stock_pos] = str(in_day_high)
        
    in_search_for = '"low52":"(.+?)"'
    in_52wk_low = search_stock_details(in_search_for,in_search_string)
    #print("52 Week Low " + str(in_52wk_low))
    Low_52wk[stock_pos] = str(in_52wk_low)
    
    in_search_for = '"high52":"(.+?)"'
    in_52wk_high = search_stock_details(in_search_for,in_search_string)
    #print("52 Week High " + str(in_52wk_high))
    High_52wk[stock_pos] = str(in_52wk_high)
    
    in_search_for = '"cm_adj_low_dt":"(.+?)"'
    in_date_52wk_low_rchd = search_stock_details(in_search_for,in_search_string)
    #print("Date 52 Week Low Reached " + str(in_date_52wk_low_rchd))
    Date_52wk_Low.append(str(in_date_52wk_low_rchd))
    
    in_search_for = '"cm_adj_high_dt":"(.+?)"'
    in_date_52wk_high_rchd = search_stock_details(in_search_for,in_search_string)
    #print("Date 52 Week High Reached " + str(in_date_52wk_high_rchd))
    Date_52wk_High.append(str(in_date_52wk_high_rchd))
    
    in_search_for = '"faceValue":"(.+?)"'
    in_face_value = search_stock_details(in_search_for,in_search_string)
    #print("Face Value " + str(in_face_value))
    Face_Value.append(str(in_face_value))

In [ ]:
def get_stock_details(in_stock_name,stock_pos):
# get the stock details from NSE India website and provide link to the page

# replace the '&' with '%26'. But not required for the HTML page view source search
    in_stock_name_modify = in_stock_name.replace("&","%26")
    print(in_stock_name_modify)
    
# connect the NSE India website and read the data on html page
    in_url = 'https://www.nseindia.com/live_market/dynaContent/live_watch/get_quote/ajaxGetQuoteJSON.jsp?symbol=' + in_stock_name_modify + '&series=EQ'
    in_ref = 'https://www.nseindia.com/live_market/dynaContent/live_watch/get_quote/GetQuote.jsp?symbol=' + in_stock_name_modify
    NSE_Headers = {
           'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/64.0.3282.140 Safari/537.36 Edge/18.17763',
           'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
           'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
           'Accept-Encoding': 'gzip, deflate, br',
           'Accept-Language': 'en-US,en;q=0.8',
           'Connection': 'keep-alive',
           'referer' : in_ref
          }
    in_html_data_str = requests.get(in_url, headers = NSE_Headers).json()['data'][0]
    #print(in_html_data_str)
    
# move the values into the respective lists/columns
    in_company_name = in_html_data_str['companyName']
    #print("Company Name " + str(in_company_name))
    Company_Name.append(str(in_company_name))

    in_trade_price = in_html_data_str['lastPrice']
    #print("Last Traded Price " + str(in_trade_price))
    Last_Traded_Values[stock_pos] = str(in_trade_price)
    
    in_day_change = in_html_data_str['change']
    #print("Today's Change " + str(in_day_change))
    Change[stock_pos] = str(in_day_change)
    
    in_change_percent = in_html_data_str['pChange']
    #print("Today's Change % " + str(in_change_percent))
    Change_Percent[stock_pos] = str(in_change_percent)
    
    in_prev_close = in_html_data_str['previousClose']
    #print("Previous Day's Close " + str(in_prev_close))
    Prev_Day_Close.append(str(in_prev_close))
    
    in_open_price = in_html_data_str['open']
    #print("Open Price " + str(in_open_price))
    Open_Price[stock_pos] = str(in_open_price)
    
    in_day_low = in_html_data_str['dayLow']
    #print("Day's Low " + str(in_day_low))
    Day_Low[stock_pos] = str(in_day_low)
    
    in_day_high = in_html_data_str['dayHigh']
    #print("Day's High " + str(in_day_high))
    Day_High[stock_pos] = str(in_day_high)
        
    in_52wk_low = in_html_data_str['low52']
    #print("52 Week Low " + str(in_52wk_low))
    Low_52wk[stock_pos] = str(in_52wk_low)
    
    in_52wk_high = in_html_data_str['high52']
    #print("52 Week High " + str(in_52wk_high))
    High_52wk[stock_pos] = str(in_52wk_high)
    
    in_date_52wk_low_rchd = in_html_data_str['cm_adj_low_dt']
    #print("Date 52 Week Low Reached " + str(in_date_52wk_low_rchd))
    Date_52wk_Low.append(str(in_date_52wk_low_rchd))
    
    in_date_52wk_high_rchd = in_html_data_str['cm_adj_high_dt']
    #print("Date 52 Week High Reached " + str(in_date_52wk_high_rchd))
    Date_52wk_High.append(str(in_date_52wk_high_rchd))
    
    in_face_value = in_html_data_str['faceValue']
    #print("Face Value " + str(in_face_value))
    Face_Value.append(str(in_face_value))


In [ ]:
def get_stock_details(in_stock_name,stock_pos):
# get the stock details from NSE India website and provide link to the page

    print(in_stock_name)
    
# connect the NSE India website and read the data on html page
    tries = 5
    for t in range(tries):
        try:
            in_html_data_str = nse.get_quote(in_stock_name)
        except JSONDecodeError as j:
            print(j)
            if i < tries - 1: # i is zero indexed
                print('tries = ' + tries)
                get_stock_details(in_stock_name,stock_pos)
            else:
                raise
        break

#    in_html_data_str = nse.get_quote(in_stock_name)
    #print(in_html_data_str)
    
# move the values into the respective lists/columns
    in_company_name = in_html_data_str['companyName']
    #print("Company Name " + str(in_company_name))
    Company_Name.append(str(in_company_name))

    in_trade_price = in_html_data_str['lastPrice']
    #print("Last Traded Price " + str(in_trade_price))
    Last_Traded_Values[stock_pos] = str(in_trade_price)
    
    in_day_change = in_html_data_str['change']
    #print("Today's Change " + str(in_day_change))
    Change[stock_pos] = str(in_day_change)
    
    in_change_percent = in_html_data_str['pChange']
    #print("Today's Change % " + str(in_change_percent))
    Change_Percent[stock_pos] = str(in_change_percent)
    
    in_prev_close = in_html_data_str['previousClose']
    #print("Previous Day's Close " + str(in_prev_close))
    Prev_Day_Close.append(str(in_prev_close))
    
    in_open_price = in_html_data_str['open']
    #print("Open Price " + str(in_open_price))
    Open_Price[stock_pos] = str(in_open_price)
    
    in_day_low = in_html_data_str['dayLow']
    #print("Day's Low " + str(in_day_low))
    Day_Low[stock_pos] = str(in_day_low)
    
    in_day_high = in_html_data_str['dayHigh']
    #print("Day's High " + str(in_day_high))
    Day_High[stock_pos] = str(in_day_high)
        
    in_52wk_low = in_html_data_str['low52']
    #print("52 Week Low " + str(in_52wk_low))
    Low_52wk[stock_pos] = str(in_52wk_low)
    
    in_52wk_high = in_html_data_str['high52']
    #print("52 Week High " + str(in_52wk_high))
    High_52wk[stock_pos] = str(in_52wk_high)
    
    in_date_52wk_low_rchd = in_html_data_str['cm_adj_low_dt']
    #print("Date 52 Week Low Reached " + str(in_date_52wk_low_rchd))
    Date_52wk_Low.append(str(in_date_52wk_low_rchd))
    
    in_date_52wk_high_rchd = in_html_data_str['cm_adj_high_dt']
    #print("Date 52 Week High Reached " + str(in_date_52wk_high_rchd))
    Date_52wk_High.append(str(in_date_52wk_high_rchd))
    
    in_face_value = in_html_data_str['faceValue']
    #print("Face Value " + str(in_face_value))
    Face_Value.append(str(in_face_value))
